# Wrangle Data

Script assumes 8 files:
- ORD weather data
- ORD taxi data
- ORD flight data
- ORD seats data
- Airline fleet matrices (planes in service/seat counts)
- Tail number data (manufacturer, model or planes)
- FAA aircraft indexes (table that matches FAA plane index to FAA plane name)
- Plane names matching file (table matching plane models to FAA plane names)

Weather Variables:
https://mesonet.agron.iastate.edu/request/download.phtml?network=IL_ASOS

Flight Variables:
https://www.transtats.bts.gov/Fields.asp?Table_ID=236

## Load Packages

In [91]:
import pandas as pd
import numpy as np
import re
import math

## Load Data
Loaded in same order as above

In [92]:
ORD_weather = pd.read_csv("../data/ORD_weather.txt", sep =  ",", skiprows = 5)
ORD_outbound = pd.read_csv("../data/ORD_outbound.csv")
ORD_OTP = pd.read_csv("../data/ORD_OTP.csv")
ORD_seats = pd.read_csv('../data/ORD_seats.csv')
seat_counts_wiki = pd.read_csv("../data/seat_counts_wiki.csv")
tailnums = pd.read_csv("../data/tailnums.csv", usecols=['tail_num', 'manufacturer', 'model'])
aircraft_types = pd.read_csv("../data/L_AIRCRAFT_TYPE.csv_")
plane_names = pd.read_csv("../data/plane_names.csv")

/home/dkruszew/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (9,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Preliminary Exploratory Data Analysis

In [93]:
tails_list = tailnums.tail_num.unique().tolist()
print("Percent of ORD_OTP flights with tail number:", round((ORD_OTP.shape[0] - ORD_OTP[~ORD_OTP['TAIL_NUM'].isin(tails_list)].shape[0])/ORD_OTP.shape[0], 3))

Percent of ORD_OTP flights with tail number: 0.751


#### ORD_OTP Airline Distribution:

In [94]:
ORD_OTP['OP_UNIQUE_CARRIER'].value_counts()

UA    498026
AA    426313
OO    377365
MQ    366615
EV    249135
DL     62054
NK     52959
YX     35096
F9     19774
US     18059
B6     17369
AS     16576
YV      9572
9E      8305
VX      7939
OH      2818
Name: OP_UNIQUE_CARRIER, dtype: int64

In [95]:
print("Unique Carriers:", ORD_OTP['OP_UNIQUE_CARRIER'].unique().shape[0])

Unique Carriers: 16


#### Plane Manufacturer Distribution:

In [96]:
pd.DataFrame(tailnums['manufacturer'].value_counts()).head(20)

,manufacturer
BOEING,1560
AIRBUS,765
BOMBARDIER INC,689
EMBRAER,472
AIRBUS INDUSTRIE,409
EMBRAER S A,230
MCDONNELL DOUGLAS,117
MCDONNELL DOUGLAS AIRCRAFT CO,73
CESSNA,36
EMBRAER-EMPRESA BRASILEIRA DE,19


In [97]:
print("Unique Manufacturers:", tailnums['manufacturer'].unique().shape[0])

Unique Manufacturers: 69


Manufacturers in Wikipedia data:

In [98]:
pd.DataFrame(seat_counts_wiki.aircraft.str.split(" ", expand=True)[0].unique())

,0
0,Bombardier
1,Airbus
2,Boeing
3,Embraer
4,McDonnell


#### Plane Model Distribution:

In [99]:
pd.DataFrame(tailnums['model'].value_counts()).head(20)

,model
CL-600-2B19,337
A320-232,318
737-823,278
ERJ 170-200 LR,227
CL-600-2C10,200
A321-231,192
EMB-145LR,164
CL-600-2D24,155
737-824,135
737-924ER,130


In [100]:
print("Unique Models:", tailnums['model'].unique().shape[0])

Unique Models: 202


In [101]:
print("Number of models in FAA:", aircraft_types.shape[0])

Number of models in FAA: 425


## Wrangle weather data

In [102]:
# Remove unnecessary columns
ORD_weather = ORD_weather.drop(columns=['station', 'lon', 'lat', 'drct', 'alti', 'mslp', 'gust', 'skyc4', 'skyl4', 'wxcodes', 'ice_accretion_1hr', 'ice_accretion_3hr', 'ice_accretion_6hr', 'peak_wind_gust', 'peak_wind_drct', 'peak_wind_time', 'metar'])
# Obtain datetime
ORD_weather = ORD_weather.rename(columns={'valid' : 'date'})
ORD_weather['date'] = pd.to_datetime(ORD_weather['date'])
ORD_weather = ORD_weather.set_index("date")
# Change missing values to nan
ORD_weather = ORD_weather.replace('M', np.nan)
# Change columns data types so that averaging can occur
numeric_weather_features = ['tmpf', 'dwpf', 'relh', 'sknt', 'p01i', 'vsby', 'skyl1', 'skyl2', 'skyl3', 'feel']
categorical_weather_features = ['skyc1', 'skyc2', 'skyc3']
ORD_weather[numeric_weather_features] = ORD_weather[numeric_weather_features].apply(pd.to_numeric)
ORD_weather[categorical_weather_features] = ORD_weather[categorical_weather_features].astype('category')

Weather categorical:
- CLR = Clear
- FEW = Few
- SCT = Scattered
- BKN = Broken
- OVC = Overcast
- VV = Sensor Error (Unable to detect cloud boundaries)

In [103]:
# Strip whitespaces
ORD_weather['skyc1'] = ORD_weather['skyc1'].str.strip()
ORD_weather['skyc2'] = ORD_weather['skyc2'].str.strip()
ORD_weather['skyc3'] = ORD_weather['skyc3'].str.strip()
# Convert categories to ordinal
cloud_to_ordinal = {'CLR':0, 'FEW':1, 'SCT':2, 'BKN':3, 'OVC':5, 'VV':0}
ORD_weather.replace(cloud_to_ordinal, inplace=True)
# Take sky condition as most conservative
ORD_weather[['skyc1', 'skyc2', 'skyc3']] = ORD_weather[['skyc1', 'skyc2', 'skyc3']].fillna(value=0)
ORD_weather[['skyc1', 'skyc2', 'skyc3']] = ORD_weather[['skyc1', 'skyc2', 'skyc3']].apply(pd.to_numeric)
ORD_weather['skyc'] = ORD_weather.apply(lambda x: max(x.skyc1, x.skyc2, x.skyc3), axis=1)

In [104]:
# Strip out time categories
ORD_weather['year'] = ORD_weather.index.year
ORD_weather['month'] = ORD_weather.index.month
ORD_weather['day'] = ORD_weather.index.day
ORD_weather['hour'] = ORD_weather.index.hour
# Average columns by hour
ORD_weather_hourly = ORD_weather.groupby(['year', 'month', 'day', 'hour']).mean().reset_index()

In [105]:
ORD_weather_hourly = ORD_weather_hourly.drop(columns=['skyc1', 'skyc2', 'skyc3', 'skyl1', 'skyl2', 'skyl3'])

In [106]:
ORD_weather_hourly.head(5)

,year,month,day,hour,tmpf,dwpf,relh,sknt,p01i,vsby,feel,skyc
0,2013,1,1,0,24.98,17.96,74.290000,9.000000,0.0,9.000000,14.780000,3.000000
1,2013,1,1,1,24.89,17.78,73.995000,9.000000,0.0,8.000000,14.670000,5.000000
2,2013,1,1,2,21.20,14.00,73.320000,10.000000,0.0,9.000000,9.420000,5.000000
3,2013,1,1,3,21.14,12.14,67.723333,10.666667,0.0,9.666667,8.976667,4.333333
4,2013,1,1,4,19.94,10.94,67.580000,10.000000,0.0,10.000000,7.840000,5.000000


## Wrangle Taxi Data

In [107]:
# Remove unnecessary columns
ORD_outbound = ORD_outbound.drop(columns=['Unnamed: 0', 'pickup_community_area'])

In [108]:
ORD_outbound.head(5)

,year,month,day,hour,rides
0,2013,1,1,0,22
1,2013,1,1,1,9
2,2013,1,1,2,11
3,2013,1,1,3,3
4,2013,1,1,4,5


## Wrangle Flight Data

In [109]:
# Remove unnecessary columns
ORD_OTP = ORD_OTP.drop(columns=['Unnamed: 0'])
# Obtain datetime
ORD_OTP['FL_DATE'] = pd.to_datetime(ORD_OTP['FL_DATE'])

In [110]:
print("Number of flights without a flight time:", sum(ORD_OTP['ARR_TIME'].isna()))

Number of flights without a flight time: 66981


In [111]:
# Strip out time categories
ORD_OTP['year'] = pd.DatetimeIndex(ORD_OTP['FL_DATE']).year
ORD_OTP['month'] = pd.DatetimeIndex(ORD_OTP['FL_DATE']).month
ORD_OTP['day'] = pd.DatetimeIndex(ORD_OTP['FL_DATE']).day
# Drop out NAs in arrival time so that hour can be pulled out
ORD_OTP = ORD_OTP.dropna(subset=['ARR_TIME'])
# Extract flight hour 
ORD_OTP['hour'] = (ORD_OTP['ARR_TIME']/100).apply(math.floor)
ORD_OTP = ORD_OTP.astype({'hour': 'int64'})

In [112]:
ORD_OTP.head(5)

,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,ARR_TIME,ARR_DELAY,Unnamed: 15,year,month,day,hour
0,1,2016-06-06,AA,N3CVAA,343,12953,1295302,31703,LGA,13930,1393004,30977,ORD,1736.0,-9.0,NaN,2016,6,6,17
1,1,2016-06-13,AA,N3LCAA,343,12953,1295302,31703,LGA,13930,1393004,30977,ORD,1800.0,15.0,NaN,2016,6,13,18
2,1,2016-06-20,AA,N3ETAA,343,12953,1295302,31703,LGA,13930,1393004,30977,ORD,1721.0,-24.0,NaN,2016,6,20,17
3,1,2016-06-27,AA,N3FGAA,343,12953,1295302,31703,LGA,13930,1393004,30977,ORD,1733.0,-12.0,NaN,2016,6,27,17
4,1,2016-06-06,AA,N3JXAA,345,12953,1295302,31703,LGA,13930,1393004,30977,ORD,1813.0,-26.0,NaN,2016,6,6,18


**Debug missing months - Jarome Looking into**

In [113]:
ORD_OTP[(ORD_OTP['FL_DATE'] > '2017-04-01') & (ORD_OTP['FL_DATE'] < '2017-04-30')]['FL_DATE'].unique()

array(['2017-04-10T00:00:00.000000000', '2017-04-03T00:00:00.000000000',
       '2017-04-17T00:00:00.000000000', '2017-04-24T00:00:00.000000000',
       '2017-04-11T00:00:00.000000000', '2017-04-04T00:00:00.000000000',
       '2017-04-18T00:00:00.000000000', '2017-04-25T00:00:00.000000000',
       '2017-04-05T00:00:00.000000000', '2017-04-12T00:00:00.000000000',
       '2017-04-19T00:00:00.000000000', '2017-04-26T00:00:00.000000000',
       '2017-04-06T00:00:00.000000000', '2017-04-13T00:00:00.000000000',
       '2017-04-20T00:00:00.000000000', '2017-04-27T00:00:00.000000000',
       '2017-04-07T00:00:00.000000000', '2017-04-14T00:00:00.000000000',
       '2017-04-21T00:00:00.000000000', '2017-04-28T00:00:00.000000000',
       '2017-04-08T00:00:00.000000000', '2017-04-15T00:00:00.000000000',
       '2017-04-22T00:00:00.000000000', '2017-04-29T00:00:00.000000000',
       '2017-04-02T00:00:00.000000000', '2017-04-09T00:00:00.000000000',
       '2017-04-16T00:00:00.000000000', '2017-04-23

In [114]:
ORD_OTP[(ORD_OTP['FL_DATE'] > '2016-01-01') & (ORD_OTP['FL_DATE'] < '2017-01-01')]['FL_DATE'].unique().shape

(365,)

In [115]:
ORD_OTP[(ORD_OTP['FL_DATE'] > '2014-01-01') & (ORD_OTP['FL_DATE'] < '2015-01-01')].month.value_counts().sort_index()

1     17358
2     18291
3     22061
4     22695
5     23417
6     23865
7     25358
8     24754
9     23122
10    25304
11    23123
12    23100
Name: month, dtype: int64

## Wrangle Seat Data - FAA

Code to create start draft of plane_names.csv:  
```
tailnums.drop(columns=['tail_num']).drop_duplicates().to_csv('../data/plane_names.csv')
````

In [116]:
plane_names.head(5)

,manufacturer,model,description
0,3D ROBOTICS,SOLO,NaN
1,AERO COMMANDER,500 S,Aero Commander (500/600 Series Excpt 680FL)
2,AERO COMMANDER,500-B,Aero Commander (500/600 Series Excpt 680FL)
3,AGUSTA SPA,A109E,NaN
4,AGUSTA SPA,A119,Agusta A-119 Koala


In [117]:
# Fix duplicate manufacturer names
names = ['AIRBUS', 'CANADAIR', 'DASSAULT', 'DIAMOND AIRCRAFT', 'EMBRAER', 'GATES LEARJET', 'MCDONNELL DOUGLAS', 'PIPER', 'RAYTHEON', 'ROBINSON HELICOPTER']
for name in names:
    tailnums['manufacturer'].replace(regex=True,inplace=True,to_replace=rf'.*{name}.*',value=rf'{name}')
# Add FAA descriptions
tailnums = pd.merge(tailnums, plane_names, how='left', on=['manufacturer', 'model'])

In [118]:
tailnums.head(5)

,tail_num,manufacturer,model,description
0,N582AA,MCDONNELL DOUGLAS,DC-9-82(MD-82),McDonnell Douglas DC9 Super 80/MD81/82/83/88
1,N439AA,MCDONNELL DOUGLAS,DC-9-83(MD-83),McDonnell Douglas DC9 Super 80/MD81/82/83/88
2,N553AA,MCDONNELL DOUGLAS,DC-9-82(MD-82),McDonnell Douglas DC9 Super 80/MD81/82/83/88
3,N558AA,MCDONNELL DOUGLAS,DC-9-82(MD-82),McDonnell Douglas DC9 Super 80/MD81/82/83/88
4,N536AA,AMERICAN AIRCRAFT INC,FALCON XP,NaN


In [119]:
ORD_seats.head(5)

,Unnamed: 0,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,SEATS,PASSENGERS,UNIQUE_CARRIER,UNIQUE_CARRIER_NAME,ORIGIN_AIRPORT_ID,ORIGIN,DEST_AIRPORT_ID,DEST,AIRCRAFT_TYPE,MONTH,Unnamed: 12
0,2883,0.0,1.0,9.0,7.0,14Q,London Air Services Limited,12003,GTF,13930,ORD,686,1,NaN
1,2899,0.0,1.0,0.0,0.0,LH,Lufthansa German Airlines,10397,ATL,13930,ORD,740,1,NaN
2,2908,0.0,18.0,0.0,0.0,CX,Cathay Pacific Airways Ltd.,10299,ANC,13930,ORD,820,1,NaN
3,2915,0.0,12.0,0.0,0.0,CX,Cathay Pacific Airways Ltd.,12478,JFK,13930,ORD,820,1,NaN
4,2949,0.0,1.0,235.0,48.0,09Q,"Swift Air, LLC d/b/a Eastern Air Lines d/b/a E...",11066,CMH,13930,ORD,625,1,NaN


In [120]:
aircraft_types.head(5)

,Code,Description
0,7,Aero Commander 200
1,8,Aero Macchi AL-60
2,9,Aeronca 7-AC
3,10,Beech Bonanza 35A/C/D/E/G/H/J/K/S/V/ 36A
4,20,Bellanca CH-300


In [121]:
# Merge plane descriptions onto FAA seat data
ORD_seats_types = pd.merge(ORD_seats, aircraft_types, how='left', left_on='AIRCRAFT_TYPE', right_on='Code')

In [122]:
ORD_seats_types.head(5)

,Unnamed: 0,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,SEATS,PASSENGERS,UNIQUE_CARRIER,UNIQUE_CARRIER_NAME,ORIGIN_AIRPORT_ID,ORIGIN,DEST_AIRPORT_ID,DEST,AIRCRAFT_TYPE,MONTH,Unnamed: 12,Code,Description
0,2883,0.0,1.0,9.0,7.0,14Q,London Air Services Limited,12003,GTF,13930,ORD,686,1,NaN,686,Learjet45
1,2899,0.0,1.0,0.0,0.0,LH,Lufthansa German Airlines,10397,ATL,13930,ORD,740,1,NaN,740,McDonnell Douglas MD-11
2,2908,0.0,18.0,0.0,0.0,CX,Cathay Pacific Airways Ltd.,10299,ANC,13930,ORD,820,1,NaN,820,Boeing 747-400F
3,2915,0.0,12.0,0.0,0.0,CX,Cathay Pacific Airways Ltd.,12478,JFK,13930,ORD,820,1,NaN,820,Boeing 747-400F
4,2949,0.0,1.0,235.0,48.0,09Q,"Swift Air, LLC d/b/a Eastern Air Lines d/b/a E...",11066,CMH,13930,ORD,625,1,NaN,625,Boeing 767-200/ER/EM


Code to create L_AIRCRAFT_TYPE_with_seats.csv:  
```
aircraft_types_seats = combined_ord_seats_types[['DEPARTURES_PERFORMED', 'SEATS', 'Description']].groupby('Description').sum()  
aircraft_types_seats['plane_seats'] = aircraft_types_seats.apply(lambda row: row.SEATS/row.DEPARTURES_PERFORMED, axis = 1)
aircraft_types_seats[['plane_seats']].to_csv("../data/L_AIRCRAFT_TYPE_with_seats.csv")
```

In [123]:
# Find seat counts per carrier plane by dividing total seats of a given plane model by the number of flights it had
seat_counts_FAA = ORD_seats_types.query('DEPARTURES_PERFORMED > 0.0').groupby(['UNIQUE_CARRIER','Description']).sum().reset_index()
seat_counts_FAA['seats'] = seat_counts_FAA.apply(lambda row: row.SEATS/row.DEPARTURES_PERFORMED, axis = 1)
seat_counts_FAA = seat_counts_FAA[['UNIQUE_CARRIER', 'Description', 'seats']]

In [124]:
seat_counts_FAA.head(5)

,UNIQUE_CARRIER,Description,seats
0,04Q,Cessna Citation X Model 650/550B/550XL,8.000000
1,09Q,Boeing 737-300,142.100000
2,09Q,Boeing 737-400,127.513924
3,09Q,Boeing 737-800,165.600000
4,09Q,Boeing 767-200/ER/EM,195.100000


## Wrangle Seat Data - Wikipedia

In [125]:
seat_counts_wiki = seat_counts_wiki.drop(columns=['Unnamed: 0'])

In [126]:
seat_counts_wiki.head(5)

,airline,aircraft,in_service,pass_count
0,9E,Bombardier CRJ-200,42,50.0
1,9E,Bombardier CRJ-700,14,69.0
2,9E,Bombardier CRJ-900,3,70.0
3,9E,Bombardier CRJ-900,116,76.0
4,AA,Airbus A319-100,133,128.0


In [127]:
def wavg(data, avg_name, weight_name):
    """
    Returns weighted average
    http://stackoverflow.com/questions/10951341/pandas-dataframe-aggregate-function-using-multiple-columns
    """
    d = data[avg_name]
    w = data[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

In [128]:
# Apply weighted average function to grouped data
avg_airline_seats = pd.DataFrame({'seat_count': seat_counts_wiki.groupby('airline').apply(wavg, 'pass_count', 'in_service')})

In [129]:
avg_airline_seats.head(5)

,seat_count
airline,
9E,69.097143
AA,170.144491
AS,162.337607
B6,149.780303
DL,183.838424


## Merge data together

In [130]:
# Merge tail data onto flight data
ORD_OTP_with_tails = pd.merge(ORD_OTP, tailnums, how='left', left_on=['TAIL_NUM'], right_on=['tail_num'])

In [131]:
ORD_OTP_with_tails.tail(5)

,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,DEST_AIRPORT_ID,...,ARR_DELAY,Unnamed: 15,year,month,day,hour,tail_num,manufacturer,model,description
2100989,7,2019-02-24,OO,N679SA,5892,12339,1233904,32337,IND,13930,...,28.0,NaN,2019,2,24,13,N679SA,BOMBARDIER INC,CL-600-2B19,Canadair RJ-200ER /RJ-440
2100990,7,2019-02-24,OO,N200SY,5906,10693,1069302,30693,BNA,13930,...,65.0,NaN,2019,2,24,10,N200SY,EMBRAER,ERJ 170-200 LR,Embraer-Emb-170
2100991,7,2019-02-24,OO,N782SK,5921,10397,1039707,30397,ATL,13930,...,3.0,NaN,2019,2,24,16,N782SK,BOMBARDIER INC,CL-600-2C10,Canadair RJ-700
2100992,7,2019-02-24,OO,N110SY,5958,11433,1143302,31295,DTW,13930,...,77.0,NaN,2019,2,24,11,N110SY,EMBRAER,ERJ 170-200 LR,Embraer-Emb-170
2100993,7,2019-02-24,OO,N207SY,5995,14683,1468305,33214,SAT,13930,...,14.0,NaN,2019,2,24,20,N207SY,EMBRAER,ERJ 170-200 LR,Embraer-Emb-170


In [132]:
# Merge seat data onto flights that have tails
ORD_OTP_with_tails_seats = pd.merge(ORD_OTP_with_tails, seat_counts_FAA, how='left', left_on=['OP_UNIQUE_CARRIER', 'description'], right_on=['UNIQUE_CARRIER', 'Description'])

In [133]:
ORD_OTP_with_tails_seats.tail(5)

,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,DEST_AIRPORT_ID,...,month,day,hour,tail_num,manufacturer,model,description,UNIQUE_CARRIER,Description,seats
2100989,7,2019-02-24,OO,N679SA,5892,12339,1233904,32337,IND,13930,...,2,24,13,N679SA,BOMBARDIER INC,CL-600-2B19,Canadair RJ-200ER /RJ-440,OO,Canadair RJ-200ER /RJ-440,50.000000
2100990,7,2019-02-24,OO,N200SY,5906,10693,1069302,30693,BNA,13930,...,2,24,10,N200SY,EMBRAER,ERJ 170-200 LR,Embraer-Emb-170,NaN,NaN,NaN
2100991,7,2019-02-24,OO,N782SK,5921,10397,1039707,30397,ATL,13930,...,2,24,16,N782SK,BOMBARDIER INC,CL-600-2C10,Canadair RJ-700,OO,Canadair RJ-700,69.011412
2100992,7,2019-02-24,OO,N110SY,5958,11433,1143302,31295,DTW,13930,...,2,24,11,N110SY,EMBRAER,ERJ 170-200 LR,Embraer-Emb-170,NaN,NaN,NaN
2100993,7,2019-02-24,OO,N207SY,5995,14683,1468305,33214,SAT,13930,...,2,24,20,N207SY,EMBRAER,ERJ 170-200 LR,Embraer-Emb-170,NaN,NaN,NaN


In [149]:
print("Seat data added for:", round((-ORD_OTP_with_tails_seats.seats.isna()).sum()*100/ORD_OTP_with_tails_seats.shape[0]), "% of rows")

Seat data added for: 69.0 % of rows


In [135]:
# Impute remaining seat NaNs with average for airline
ORD_OTP_with_tails_seats_and_imputation = pd.merge(ORD_OTP_with_tails_seats, avg_airline_seats.reset_index(), how='left', left_on=['OP_UNIQUE_CARRIER'], right_on=['airline'])
ORD_OTP_with_tails_seats_and_imputation['seats'] = (-ORD_OTP_with_tails_seats_and_imputation.seats.isna() * ORD_OTP_with_tails_seats_and_imputation.seats).fillna(0) + (ORD_OTP_with_tails_seats_and_imputation.seats.isna() * ORD_OTP_with_tails_seats_and_imputation.seat_count)
ORD_OTP_with_tails_seats_and_imputation = ORD_OTP_with_tails_seats_and_imputation.drop(columns=['seat_count'])

In [136]:
ORD_OTP_with_tails_seats_and_imputation.head()

,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,DEST_AIRPORT_ID,...,day,hour,tail_num,manufacturer,model,description,UNIQUE_CARRIER,Description,seats,airline
0,1,2016-06-06,AA,N3CVAA,343,12953,1295302,31703,LGA,13930,...,6,17,NaN,NaN,NaN,NaN,NaN,NaN,170.144491,AA
1,1,2016-06-13,AA,N3LCAA,343,12953,1295302,31703,LGA,13930,...,13,18,NaN,NaN,NaN,NaN,NaN,NaN,170.144491,AA
2,1,2016-06-20,AA,N3ETAA,343,12953,1295302,31703,LGA,13930,...,20,17,NaN,NaN,NaN,NaN,NaN,NaN,170.144491,AA
3,1,2016-06-27,AA,N3FGAA,343,12953,1295302,31703,LGA,13930,...,27,17,NaN,NaN,NaN,NaN,NaN,NaN,170.144491,AA
4,1,2016-06-06,AA,N3JXAA,345,12953,1295302,31703,LGA,13930,...,6,18,NaN,NaN,NaN,NaN,NaN,NaN,170.144491,AA


Note:  
Alternative Imputation using apply (takes 2 minutes)  
`ORD_OTP_with_tails_seats.apply(lambda x: avg_airline_seats.loc[x.OP_UNIQUE_CARRIER].seat_count if pd.isna(x.seats) else x.seats, axis=1)`

In [137]:
# Get seats and flights per hour
seats_per_hour = ORD_OTP_with_tails_seats_and_imputation.groupby(['year','month','day','hour']).sum().reset_index()[['year','month','day','hour','seats']]
flights_per_hour = ORD_OTP_with_tails_seats_and_imputation.groupby(['year','month','day','hour']).count().reset_index()[['year','month','day','hour','airline']]

In [138]:
seats_per_hour.head(5)

,year,month,day,hour,seats
0,2013,1,1,0,464.876254
1,2013,1,1,1,147.962430
2,2013,1,1,4,847.150186
3,2013,1,1,5,1306.117805
4,2013,1,1,6,1245.080703


In [139]:
flights_per_hour.head(5)

,year,month,day,hour,airline
0,2013,1,1,0,3
1,2013,1,1,1,1
2,2013,1,1,4,4
3,2013,1,1,5,9
4,2013,1,1,6,19


In [140]:
# Merge taxi and weather data
ORD_outbound_weather = pd.merge(ORD_outbound, ORD_weather_hourly, how='left', on=['year', 'month', 'day', 'hour'])

In [141]:
ORD_outbound_weather.head(5)

,year,month,day,hour,rides,tmpf,dwpf,relh,sknt,p01i,vsby,feel,skyc
0,2013,1,1,0,22,24.98,17.96,74.290000,9.000000,0.0,9.000000,14.780000,3.000000
1,2013,1,1,1,9,24.89,17.78,73.995000,9.000000,0.0,8.000000,14.670000,5.000000
2,2013,1,1,2,11,21.20,14.00,73.320000,10.000000,0.0,9.000000,9.420000,5.000000
3,2013,1,1,3,3,21.14,12.14,67.723333,10.666667,0.0,9.666667,8.976667,4.333333
4,2013,1,1,4,5,19.94,10.94,67.580000,10.000000,0.0,10.000000,7.840000,5.000000


In [142]:
# Merge taxi/weather to seats/flights
ORD_outbound_weather_seats = pd.merge(ORD_outbound_weather, seats_per_hour, how='left', on=['year', 'month', 'day', 'hour'])
ORD_outbound_weather_seats_flights = pd.merge(ORD_outbound_weather_seats, flights_per_hour, how='left', on=['year', 'month', 'day', 'hour'])

In [143]:
ORD_outbound_weather_seats_flights

,year,month,day,hour,rides,tmpf,dwpf,relh,sknt,p01i,vsby,feel,skyc,seats,airline
0,2013,1,1,0,22,24.98,17.96,74.290000,9.000000,0.0000,9.000000,14.780000,3.000000,464.876254,3.0
1,2013,1,1,1,9,24.89,17.78,73.995000,9.000000,0.0000,8.000000,14.670000,5.000000,147.962430,1.0
2,2013,1,1,2,11,21.20,14.00,73.320000,10.000000,0.0000,9.000000,9.420000,5.000000,NaN,NaN
3,2013,1,1,3,3,21.14,12.14,67.723333,10.666667,0.0000,9.666667,8.976667,4.333333,NaN,NaN
4,2013,1,1,4,5,19.94,10.94,67.580000,10.000000,0.0000,10.000000,7.840000,5.000000,847.150186,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61977,2020,1,31,20,257,37.00,30.00,75.560000,5.076923,0.0001,7.615385,31.550000,5.000000,4615.161518,41.0
61978,2020,1,31,21,243,37.00,30.00,75.560000,5.153846,0.0000,7.615385,31.550000,5.000000,4305.418873,37.0
61979,2020,1,31,22,154,37.00,30.00,75.560000,4.714286,0.0000,7.000000,32.330000,5.000000,1997.138157,13.0
61980,2020,1,31,23,77,36.00,28.90,75.150000,5.076923,0.0000,8.615385,30.350000,5.000000,2122.022524,12.0


## Booyah

In [150]:
ORD_outbound_weather_seats_flights.to_csv('../data/ORD.csv')